# Adam Ledet
**Github Page**: https://github.com/adamledet/itds-project-2025

**Website**: https://adamledet.github.io/itds-project-2025/

My first few code blocks set everything up to properly manipulate the datasets. Before doing anything, I need to get my API key to use Kaggle as this is my primary dataset source. I upload the file from my computer each time rather than linking it to my Google Drive.

In [ ]:
# Install Kaggle and upload your API key.
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adamledet","key":"9f5e44bbd95759f9049ca793c04da24f"}'}

In [ ]:
# Make a Kaggle directory and give permission to edit it.
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                                    title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
atharvasoundankar/chocolate-sales                                      Chocolate Sales Data 📊🍫                              63KB  2025-03-07 05:44:03           5219         69  1.0              
abdulmalik1518/mobiles-dataset-2025                                    Mobiles Dataset (2025)                               20KB  2025-02-18 06:50:24          12132        214  1.0              
salahuddinahmedshuvo/ecommerce-consumer-behavior-analysis-data         Ecommerce Consumer Behavior Analysis Data            43KB  2025-03-03 13:09:09           2700         44  0.9411765        
willianoliveiragibin/pixa

Next I actually get the datasets I'll be using. These datasets are https://www.kaggle.com/datasets/joebeachcapital/top-1000-steam-games and https://www.kaggle.com/datasets/trolukovich/steam-games-complete-dataset; the latter of which is more robust but somewhat outdated while the former is more recent but has several flaws. The older dataset I call **df_Old** while I call the newer dataset **df_New**.

In [ ]:
# Get two datasets
import kagglehub
! kaggle datasets download 'joebeachcapital/top-1000-steam-games' # 93182 Steam Games
! kaggle datasets download 'trolukovich/steam-games-complete-dataset' # Steam games complete dataset
! mkdir steamNew
! mkdir steamOld
! unzip /content/top-1000-steam-games.zip -d steamNew
! unzip /content/steam-games-complete-dataset.zip -d steamOld

Dataset URL: https://www.kaggle.com/datasets/joebeachcapital/top-1000-steam-games
License(s): CC0-1.0
top-1000-steam-games.zip: Skipping, found more recently modified local copy (use --force to force download)
Dataset URL: https://www.kaggle.com/datasets/trolukovich/steam-games-complete-dataset
License(s): CC0-1.0
steam-games-complete-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘steamNew’: File exists
mkdir: cannot create directory ‘steamOld’: File exists
Archive:  /content/top-1000-steam-games.zip
replace steamNew/93182_steam_games.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace steamNew/93182_steam_games.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: steamNew/93182_steam_games.csv  
  inflating: steamNew/steam_app_data.csv  
  inflating: steamNew/steamspy_data.csv  
Archive:  /content/steam-games-complete-dataset.zip
  inflating: steamOld/steam_games.csv  


In [29]:
import pandas as pd
import numpy as np

In [ ]:
df_New = pd.read_csv("/content/steamNew/93182_steam_games.csv")
df_New.head()

<ipython-input-6-844d21ff1713>:1: DtypeWarning: Columns (3,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_New = pd.read_csv("/content/steamNew/93182_steam_games.csv")


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,1424640,余烬,"Oct 3, 2020",20000 - 50000,0,0,3.99,0,'Ashes of war' is an anti war theme adventure ...,['Simplified Chinese'],...,0,0,0,宁夏华夏西部影视城有限公司,宁夏华夏西部影视城有限公司,"Single-player,Family Sharing","Adventure,Casual,Indie,RPG","Sokoban,RPG,Puzzle-Platformer,Exploration,Adve...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
1,402890,Nyctophilia,"Sep 23, 2015",50000 - 100000,0,0,0.00,0,NYCTOPHILIA Nyctophilia is an 2D psychological...,"['English', 'Russian']",...,0,0,0,Cat In A Jar Games,Cat In A Jar Games,Single-player,"Adventure,Free To Play,Indie","Free to Play,Indie,Adventure,Horror,2D,Pixel G...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
2,1151740,Prison Princess,"Apr 2, 2020",0 - 20000,0,0,19.99,0,"ABOUT Now nothing more than a phantom, can the...","['English', 'Simplified Chinese', 'Traditional...",...,0,0,0,qureate,qureate,"Single-player,Steam Achievements,Full controll...","Adventure,Indie","Sexual Content,Adventure,Indie,Nudity,Anime,Ma...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
3,875530,Dead In Time,"Oct 12, 2018",0 - 20000,0,0,7.99,0,Is a hardcore action with a non-trivial level ...,"['English', 'Russian']",...,0,0,0,Zelenov Artem,Zelenov Artem,"Single-player,Full controller support,Family S...","Action,Indie","Action,Indie,Souls-like,Fantasy,Early Access,R...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
4,1835360,Panacle: Back To Wild,"Mar 11, 2022",0 - 20000,2,0,3.99,0,Panacle: Back to the Wild is a indie card game...,"['English', 'Japanese', 'Simplified Chinese', ...",...,0,0,0,渡鸦游戏,"渡鸦游戏,电钮组","Single-player,Family Sharing","Indie,Strategy,Early Access","Trading Card Game,Turn-Based Strategy,Lore-Ric...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...


In [ ]:
df_Old = pd.read_csv("/content/steamOld/steam_games.csv")
df_Old.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


At this point I can actually start manipulating my data and getting results. The first thing I want to do is look through my datasets and make a correlation matrix with all my integers. This is an initial step to test my variables and see if there are any surprising results. My initial expectation was that there would be few correlations between the integer data, and any that do appear will be by product of chance.

In [46]:
# Get correlation matrix for integer values and return matrix only with values greater than the threshold (and excluding self, which would have a correlation of 1)
# Threshold currently set at 0.4
thresh = 0.4
df_New_CorrMatrix = df_New[["Peak CCU", "Required age", "Price", "DLC count", "Metacritic score", "User score", "Positive", "Negative", "Score rank", "Achievements", "Recommendations", "Average playtime forever", "Average playtime two weeks", "Median playtime two weeks"]].corr()
df_New_CorrMatrix[df_New_CorrMatrix[df_New_CorrMatrix>thresh] < 1]

,Peak CCU,Required age,Price,DLC count,Metacritic score,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime two weeks
Peak CCU,NaN,NaN,NaN,NaN,NaN,NaN,0.715739,0.54617,NaN,NaN,NaN,NaN,NaN,NaN
Required age,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Price,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DLC count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Metacritic score,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
User score,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.842230,NaN,NaN,NaN,NaN,NaN
Positive,0.715739,NaN,NaN,NaN,NaN,NaN,NaN,0.73174,NaN,NaN,NaN,NaN,NaN,NaN
Negative,0.546170,NaN,NaN,NaN,NaN,NaN,0.731740,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Score rank,NaN,NaN,NaN,NaN,NaN,0.84223,NaN,NaN,NaN,0.590452,NaN,NaN,NaN,NaN
Achievements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.590452,NaN,NaN,NaN,NaN,NaN


In the New dataset, there are strongest correlations between:


*   Peak CCU (Peak Concurrent Users) and Positive / Negative reviews
*   Positive Reviews and Negative reviews
*   Score rank and number of achievements

It makes logical sense for there to be a correlation between positive and negative reviews with well-liked games receiving more positive and fewer negative reviews. The same is true for a popular game to have more players while more negatively reviewed games have fewer peak players. Sadly, the dataset doesn't have working data for the average playtime as these values would be interesting to see. Let's look at our other dataset next.

In [67]:
#df_Old["original_price"] = df_Old["original_price"].replace('[\$]', '', regex=True)
#df_Old["original_price"] = df_Old["original_price"].replace("Free",0)
#df_Old["original_price"] = df_Old["original_price"].replace("Free to play",0)
#df_Old["original_price"] = df_Old["original_price"].replace("Free to Play",0)
#df_Old["original_price"] = df_Old["original_price"].replace("Free To Play",0).astype(float)
#df_Old["original_price"]

ValueError: could not convert string to float: 'Demo Middle-earth™: Shadow of War™'